Following the Baselines and Bigrams: Simple, Good Sentiment and Topic Classification paper
Specifically trying MNB and NBSVM
https://www.aclweb.org/anthology/P12-2018

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.model_selection import GridSearchCV, train_test_split
import spacy
import time
nlp = spacy.load('en')

In [2]:
df = pd.read_csv('train.csv')
X_train, X_test, y_train, y_test = train_test_split(df[['comment_text']],
                                                    df.drop(columns=['id', 'comment_text']),
                                                    test_size=0.2)
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [3]:
nlp.vocab[u'you'].is_stop = False

In [4]:
def tokenize(s):
    tokens = nlp(unicode(s))
    return [t.lemma_ if t.lemma_ != u'-PRON-' else t.text for t in tokens if not t.is_stop and not t.is_space and not t.is_punct]

In [5]:
tfidf_vec = TfidfVectorizer(strip_accents='unicode', tokenizer=tokenize, ngram_range=(1,2),
                            max_df=0.9, min_df=3, sublinear_tf=True)

In [6]:
X_train_tfidf = tfidf_vec.fit_transform(X_train.comment_text)
X_test_tfidf = tfidf_vec.transform(X_test.comment_text)

In [7]:
def get_r(X, y_pos, y_neg, alpha=1.):
    p = X[y_pos].sum(0) + alpha
    q = X[y_neg].sum(0) + alpha
    
    p /= sum(p) + alpha
    q /= sum(q) + alpha
    
    return np.log(p, q).A1

In [60]:
lr = LogisticRegression()

In [61]:
search_params = {
    'tol': [1e-3, 1e-4],
    'C': [1., 4., 8., 16.]
}

In [62]:
best_params = {}
best_estimators = {}
rs = {}
best_score = {}
test_score = {}
for c in classes:
    print 'Training {}'.format(c)
    t = time.time()
    y_pos = y_train[y_train[c] == 1][c].values
    y_neg = y_train[y_train[c] == 0][c].values
    r = get_r(X_train_tfidf, y_pos, y_neg)
    clf = GridSearchCV(lr, search_params, scoring='neg_log_loss', cv=10)
    clf.fit(X_train_tfidf.multiply(r), y_train[c].values)
    best_params[c] = clf.best_params_
    best_estimators[c] = clf.best_estimator_
    rs[c] = r
    best_score[c] = clf.best_score_
    test_score[c] = log_loss(y_test[c].values, clf.best_estimator_.predict_proba(X_test_tfidf.multiply(r))[:,1])
    print 'Took {} seconds'.format(time.time() - t)

Training toxic
Took 128.82940197 seconds
Training severe_toxic
Took 139.745584965 seconds
Training obscene
Took 118.517186165 seconds
Training threat
Took 119.408869028 seconds
Training insult
Took 134.984273911 seconds
Training identity_hate
Took 138.027602911 seconds


In [63]:
print 'Mean Validation score: {}'.format(np.mean(best_score.values()))
print 'Mean Test score: {}'.format(np.mean(test_score.values()))

Mean Validation score: -0.0543013555103
Mean Test score: 0.0511423051548


In [72]:
print 'Best parameters by Class:'
for c in classes:
    print c, best_params[c]
    print 'Confusion matrix'
    print confusion_matrix(y_test[c], [1 if x > 0.5 else 0 for x in best_estimators[c].predict_proba(X_test_tfidf.multiply(rs[c]))[:,1]])

Best parameters by Class:
toxic {'C': 16.0, 'tol': 0.0001}
Confusion matrix
[[17214   173]
 [  576  1208]]
severe_toxic {'C': 8.0, 'tol': 0.0001}
Confusion matrix
[[18929    39]
 [  153    50]]
obscene {'C': 16.0, 'tol': 0.0001}
Confusion matrix
[[18060    85]
 [  330   696]]
threat {'C': 16.0, 'tol': 0.0001}
Confusion matrix
[[19111     7]
 [   40    13]]
insult {'C': 8.0, 'tol': 0.001}
Confusion matrix
[[18108   126]
 [  419   518]]
identity_hate {'C': 16.0, 'tol': 0.001}
Confusion matrix
[[19001    17]
 [  116    37]]


False Negative rate is really high likely due to severe class imbalance.